# Reading the Bosch BME280 pressure, humidity and temperature sensor

Before, we need to connect the BME280 board which uses I2C bus (4 wires : VCC, GND, SCL, SDA). See the [I2C documentation](https://docs.pycom.io/chapter/firmwareapi/pycom/machine/I2C.html#i2cinitmode---baudrate100000-pinssda-scl) and the [Expansion Board] pins(https://docs.pycom.io/chapter/datasheets/boards/expansion3.html) documentation.

We will use pins G16/GPIO16/P9 (SDA) and G17/GPIO17/P10 (SCL) on Expansion Board, so connect them via jumpers (wires) to BME280's SDA and SCL. Also VCC from BME280 to 3V3 for Expansion Board, and GND between them. 

[This BME280 MicroPython driver](https://github.com/neliogodoi/MicroPython-BME280) which has many configuration options for the BME280 sensor. Download the ['bme280.py' file](https://github.com/rcolistete/MicroPython_Tutorial_SciPy_2018/blob/master/4_Reading_BME280_sensor/bme280.py).

To copy using rshell is very simple (before entering the repl) :
```
> cp bme280.py /pyboard/flash/lib/bme280.py
```

Initialiating I2C, loading the driver, initialiating the BME280 sensor :
```Python
>>> from machine import I2C
>>> import bme280
>>> i2c = I2C(0, I2C.MASTER)
>>> i2c.scan()
[118]
>>> sensor = bme280.BME280(i2c=i2c)
>>> sensor.
pressure        __qualname__    values          read_raw_data
__module__      temperature     humidity        altitude
formated_values                 pressure_precision
read_compensated_data           __init__
>>> sensor.formated_values
('22.85 C', '99661.15 Pa', '57.79395 %')
>>> sensor.values
(22.87, 99660.63, 57.58789)
```

Try your breath to increase the humidity readings :
```Python
>>> while True:
...     print(sensor.humidity)
... 
57.73145
57.72168
57.7334
```

Some statistics (mean, variance, standard deviation), use Ctrl+E to paste in REPL :
```Python
def PressureLoopMeanStdDev(numReadings=100):
    samples = [0.0]*numReadings; mean = 0.0
    i = 0
    while (i < numReadings):
        pressure = sensor.pressure
        samples[i] = pressure
        mean += pressure
        i += 1
    mean /= numReadings
    variance = 0.0
    for pressure in samples:
        variance += (pressure - mean)**2
    variance /= (numReadings - 1)
    print("%u readings :\n%s" %(numReadings, str(samples)))
    print("Mean of pressure readings (Pa) = %15.13f" % mean)
    print("Variance of pressure readings = %15.13f" % variance)
    print("10**6*Variance/(Mean**2) of pressure readings = %15.13f" % ((variance*10**6)//(mean**2)))
    print("Standard deviation of pressure readings = %15.13f" % math.sqrt(variance))
```
math module is needed :
```Python
>>> import math
>>> PressureLoopMeanStdDev()
100 readings :
[99660.61, 99660.3, 99660.41, 99659.94, 99659.86, 99659.63, 99659.4, 99659.4, 99659.56, 99659.9, 99660.06, 99660.12, 99660.12, 99660.45, 99660.29, 99660.29, 99659.73, 99659.56, 99659.34, 99659.9, 99659.73, 99660.12, 99659.97, 99660.53, 99660.53, 99660.92, 99660.59, 99660.76, 99660.2, 99660.36, 99660.02, 99659.7, 99659.77, 99659.77, 99659.77, 99660.17, 99660.17, 99660.41, 99660.56, 99660.23, 99660.23, 99660.23, 99659.67, 99660.23, 99660.07, 99660.46, 99660.3, 99660.14, 99660.2, 99660.2, 99660.2, 99660.43, 99659.81, 99658.85, 99658.52, 99658.13, 99658.36, 99658.75, 99658.42, 99658.81, 99658.65, 99658.65, 99659.21, 99658.89, 99658.72, 99658.49, 99658.16, 99658.33, 99658.33, 99658.56, 99658.72, 99659.12, 99659.12, 99659.51, 99659.36, 99659.19, 99659.19, 99658.86, 99658.13, 99657.96, 99657.96, 99658.36, 99658.2, 99658.03, 99658.59, 99658.43, 99658.82, 99658.82, 99658.43, 99658.26, 99658.03, 99657.87, 99657.7, 99657.55, 99657.6, 99657.6, 99658.16, 99657.84, 99658.0, 99658.0]
Mean of pressure readings (Pa) = 99659.3189239501953
Variance of pressure readings = 0.8624532699585
10**6*Variance/(Mean**2) of pressure readings = 0.0000000000000
Standard deviation of pressure readings = 0.9286836624146
```